# Programming and Database project

## Cars for sale in the US: an analysis





**Student**: Gianello Alessio <br>
**ID**: VR502062

## Import and Clean of the dataset

This dataset provides comprehensive information about used cars available for sale in the United States. It includes detailed data on various aspects of each vehicle, making it a valuable resource for car buyers, sellers, and data enthusiasts. 
The dataset contains the following key attributes:
- Model: The specific model of the car.
- Mileage: The number of miles the car has been driven.
- Year: The manufacturing year of the car.
- Status: Indicates whether the car is new, used, or certified pre-owned.
- Dealer: Information about the dealer or seller offering the car.
- Price: The listed price of the car in USD.

dataset source: https://www.kaggle.com/datasets/juanmerinobermejo/us-sales-cars-dataset



Useful imports:


In [212]:
import pandas as pd

In [213]:
# Reading data
cars_list_df = pd.read_csv('cars.csv', encoding='utf-16')

In [214]:
# Taking a look at what we have at hand
cars_list_df.head(10)

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
3,Ford,Bronco,2023,Used,1551.0,Mike Smith Chrysler Dodge Jeep RAM,58900.0
4,Acura,TLX,2021,Used,30384.0,Mike Smith Nissan,34499.0
5,Volkswagen,Golf,2022,Certified,13895.0,Volkswagen of Beaumont,34000.0
6,GMC,Yukon,2021,Used,68506.0,BMW of Beaumont,56954.0
7,BMW,M340,2023,New,NaN,BMW of Beaumont,61715.0
8,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,37945.0
9,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,33495.0


In [215]:
# Taking a closer look
cars_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51793 entries, 0 to 51792
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Brand    51793 non-null  object 
 1   Model    51793 non-null  object 
 2   Year     51793 non-null  int64  
 3   Status   51793 non-null  object 
 4   Mileage  22981 non-null  float64
 5   Dealer   51689 non-null  object 
 6   Price    50644 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 2.8+ MB


Set columns name to lower case:

In [216]:
cars_list_df.columns = cars_list_df.columns.map(lambda x: x.lower())

It's evident that there are a lot of null values, mainly in column 4, more precisely:

In [217]:
#percentage of not-null values in each columns, i define a function beacuse I'll surely need to use it later

def get_null_percentage(df): 
 df_length =len(df)
 return (df.notnull().sum()/df_length)*100 

get_null_percentage(cars_list_df)


brand      100.000000
model      100.000000
year       100.000000
status     100.000000
mileage     44.370861
dealer      99.799201
price       97.781553
dtype: float64

## Managing null values

Mileage has a huge percentage of null values. It may be tempting to drop them, but getting a closer look:

In [218]:
cars_list_df[cars_list_df['status'] == 'New'] 

,brand,model,year,status,mileage,dealer,price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
7,BMW,M340,2023,New,NaN,BMW of Beaumont,61715.0
8,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,37945.0
...,...,...,...,...,...,...,...
51788,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,122455.0
51789,Chevrolet,Corvette,2023,New,NaN,Chase Chevrolet,125930.0
51790,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,53715.0
51791,Jeep,Wrangler,2023,New,NaN,Stockton Dodge Chrysler Jeep RAM,91450.0


We may think that mileage is null on New cars, let's check it:

In [219]:
cars_list_df[(cars_list_df['status'] != 'New') & (cars_list_df['mileage'].isna())] 

,brand,model,year,status,mileage,dealer,price
29127,BMW,Coupe,2023,Used,NaN,BMW of Gwinnett Place,48175.0


So we put it at 0

In [220]:
cars_list_df['mileage'].fillna(0, inplace=True) # Inplace = True edits the original df

In [221]:
cars_list_df = cars_list_df.dropna(subset=['price'])


We still need to handle the null values in the 'Dealer' field. Since this is a descriptive (and not quantitative) field, we will replace the null values with 'unknown_dealer'


In [222]:
cars_list_df['dealer'].fillna('unkown_dealer', inplace=True) # Inplace = True edits the original df

We now shouldn't have colums with null values, let's check it:

In [223]:
#percentage of not-null values in each columns

get_null_percentage(cars_list_df)

brand      100.0
model      100.0
year       100.0
status     100.0
mileage    100.0
dealer     100.0
price      100.0
dtype: float64

## Analysis of attributes and links between them

In the following sections I'll perform a .value_counts() for every descriptive column of the dataframe, to check for anomalies.

In [224]:
cars_list_df.brand.value_counts().sort_index() #Sorting index for a faster check for troubles

Acura                   819
Alfa Romeo              173
Aston Martin             71
Audi                   1493
BMW                    2422
Bentley                 103
Buick                   465
Cadillac               1150
Chevrolet              5008
Chrysler                342
Dodge                  1205
FIAT                     28
Ferrari                  47
Ford                   6383
GMC                    2581
Genesis                 589
Geo                       1
Honda                  2012
Hummer                   19
Hyundai                1129
Infiniti                553
International Scout       1
Isuzu                     1
Jaguar                  268
Jeep                   3224
Karma                     4
Kia                    1709
Lamborghini              37
Land Rover              842
Lexus                  1788
Lincoln                 587
Lotus                     3
Lucid                     4
MINI                    127
Maserati                149
Maybach             

In [225]:
def isCountDifferent(dataframe, column_name):
    indexes =dataframe[column_name].value_counts().index
    formatted_indexes = indexes.map(lambda x: x.lower().replace(' ','_').replace('-',''))
    return len(formatted_indexes)!=len(indexes)

isCountDifferent(cars_list_df, 'brand')

False

In [226]:
cars_list_df.brand.value_counts() #Sorting index for a faster check for troubles

Ford                   6383
Chevrolet              5008
Toyota                 4645
Jeep                   3224
GMC                    2581
BMW                    2422
Mercedes               2050
Honda                  2012
RAM                    1812
Lexus                  1788
Kia                    1709
Audi                   1493
Nissan                 1472
Dodge                  1205
Mazda                  1189
Cadillac               1150
Hyundai                1129
Volkswagen             1045
Subaru                  849
Land Rover              842
Acura                   819
Volvo                   695
Porsche                 683
Genesis                 589
Lincoln                 587
Infiniti                553
Buick                   465
Tesla                   351
Chrysler                342
Mitsubishi              284
Jaguar                  268
Alfa Romeo              173
Maserati                149
MINI                    127
Bentley                 103
Aston Martin        

In [227]:
isCountDifferent(cars_list_df, 'model')

False

So there aren't doubles in car models or brands caused by format problems, I can get back to my analysis

In [228]:
cars_list_df.year.value_counts().sort_index()

1959        1
1967        1
1968        1
1969        3
1970        3
1971        4
1972        2
1973        4
1975        2
1976        1
1977        5
1978        1
1979        4
1982        5
1983        1
1984        3
1985        2
1986        3
1987        9
1988        3
1989        8
1990       12
1991        4
1992        5
1993        9
1994       14
1995       18
1996       12
1997       11
1998       11
1999       26
2000       42
2001       44
2002       65
2003       58
2004      105
2005      105
2006      147
2007      182
2008      217
2009      159
2010      243
2011      371
2012      430
2013      591
2014      765
2015      986
2016     1148
2017     1493
2018     1765
2019     2280
2020     3149
2021     3345
2022     4135
2023    20466
2024     8165
Name: year, dtype: int64

It seems like most of the cars are from recent years, I'll check it later

In [229]:
cars_list_df.status.value_counts().sort_index()

Certified     3076
New          27669
Used         19899
Name: status, dtype: int64

In [230]:
cars_list_df.dealer.value_counts().sort_index()

#1 Brookville Chevrolet Buick           1
#1 Cochran Cadillac Monroeville         4
#1 Cochran Chevrolet                    2
#1 Cochran Chrysler Dodge Jeep Ram      4
#1 Cochran Ford                         3
                                     ... 
iDrive Motors                           5
iDrive1 Motorcars                       3
iGOTCARS                               27
unkown_dealer                         103
xDrive Motors Inc.                      2
Name: dealer, Length: 8513, dtype: int64

In [231]:
cars_list_df['dealer'].str.startswith('#1').value_counts()

False    50617
True        27
Name: dealer, dtype: int64

Get rid of '#1' in the dealers name and see if False goes up:

In [240]:
dealers = cars_list_df['dealer']
sus_dealers= [dealer[3:] if dealer.startswith('#1') else dealer for dealer in dealers]
normal_dealers = [dealer[3:] if not dealer.startswith('#1') else dealer for dealer in dealers]

In [236]:
len(cars_list_df['dealer'])

50644